#Modeling U.S Gun-Homcide Rates using SVI
###A comprehensive analysis by Adam Sam

<img src="https://www.bridgemi.com/sites/default/files/styles/full_width_image/public/2022-05/gun%20shutterstock.jpg?itok=AUziubj7" width="520" height="300">



##Datasets
Gun homicide: https://wonder.cdc.gov/controller/saved/D158/D429F689

SVI: https://www.atsdr.cdc.gov/place-health/php/svi/index.html

**-Are there correlations between features?**

The dataset includes multiple features grouped into four themes: Socioeconomic Status, Household Characteristics, Racial & Ethnic Minority Status, and Housing Type & Transportation.

Features like poverty (EP_POV150),unemployment (EP_UNEMP), and lack of high school diploma (EP_NOHSDP) could likely be positively correlated.

Features such as elderly population (EP_AGE65) and disability (EP_DISABL) may also be correlated.

Variables starting with "RPL_" represent the final overall social vulnerability rankings calculated based on the preceding determining variables (non-RPL variables). These are likely correlated with said variables, and will be separated.

A correlation matrix could quantitatively confirm these relationships.

**-Are there outliers?**
Washington D.C may be an outlier, as it has the highest homicide rate of 21 per 100k.

Some other interesting outliers:
EP_MINRTY (percentage who are considered minorities): Hawaii (73.76) and D.C (63.7) are much higher than the average.

EP_UNINSUR (percentage who are uninsured): Texas (17.41) and Oklahoma (15.16) are outliers compared to states like Massachusetts (3.01).

**-Will all features be used? Why or why not?**

Not all features in the dataset were used, as some were deemed as adjunct, or irrelevant for measuring social vulnerability; particularly measures of ethnicity composition.

The following variables are labeled as adjunct:

-estimate of daytime population derived from LandScan 2021 estimates

-ACS estimates for households without an internet subscription

-ACS estimates for Hispanic/Latino persons, Not Hispanic or Latino Black/African American persons, Not Hispanic or Latino Asian persons, Not
Hispanic or Latino American Indian and Alaska Native persons, Not Hispanic or Latino Native Hawaiian and Other Pacific Islander persons, Not Hispanic
or Latino persons of two or more races, and Not Hispanic or Latino persons of some other race

Variables that are not percentages were not used (raw values). Only the variables starting with "EP_" were used, as they represent percentages.

Variables starting with "RPL_" represent the final overall social vulnerability rankings calculated based on the preceding determining variables (non-RPL variables). These are likely correlated with said varaibles, and will be separated.

The RPL_THEMES feature is of main interest, as it is a ranking between 0 and 1 measuring social vulnerability considering all themes (socialeconomic, household, racial, transportation).

RPL_THEME1 to RPL_THEME4 are separate rankings for individual layers of the SVI hierarchy.

**-Is the dataset balanced?**
The dataset appears balanced in terms of geographic representation, as it includes all U.S. states and counties with consistent metrics.

However, the homicide rate per 100k varies widely (example: 1.28 in New Hampshire vs. 20.99 in D.C.), suggesting potential skewness. This could indicate imbalance if modeling homicide rates.

**-What feature engineering is needed?**

-Manually calculating homicide rate for rows where it is "Unreliable."

-Scaling the data using min=max scaling.

-Dimensionality reduction using PCA

**-Preprocessing steps**

-Grouping the data by states

-Replacing "Unreliable" values within death rate column with the proper values by manually calculating Deaths / Population * 100k.

-Irrelevant columns were removed, including:

*   Flags representing states fell within a certain category (0 or 1)
*   Extraneous state codes and abbreviations
*   Exact margin of error values
*   Miscellaneous information not used in the measurement of SVI (ethnicity composition)

##DATASET LOADING

In [174]:
""" ADAM SAM
steps to connect to lambda machine for co-lab
1. ssh -L 8888:localhost:8888 sama1@lab.cs.wit.edu -p 50004

2. jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'
--port=8888 --NotebookApp.port_retries=0

3.
On your PC, go to Google Colab and select "Connect to a local runtime" (top
right corner).

In the URL box, enter the URL with the token you received from the output of
the Jupyter Notebook command in Step 2.
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext memory_profiler

####MAKE SURE TO CD TO ML_PROJECT DIR FIRST!#####
#!cd ML_PROJECT
#scp -P 50004 "C:\Users\sama1\Downloads\f2\svi\SVI_FINAL_CLEAN_2022.csv" sama1@lab.cs.wit.edu:"~/ML_PROJECT"
#loading dataset
data = pd.read_csv("SVI_FINAL_CLEAN_2022.csv")
#data.head()

#drop "Deaths" and "Population" and "STATE"
data = data.drop(["STATE","Deaths", "Population"], axis=1)
display(data.head())

target = data.iloc[:,-1]

#display(target)

data = data.drop(columns=data.columns[-1])
display(data.head())
#separate "RPL_" columns
data_with_ranks = data.loc[:, data.columns.str.startswith('RPL_')]
data = data.loc[:, ~data.columns.str.startswith('RPL_')]

display(data_with_ranks.head())
display(data.head())


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


,EP_POV150,EP_UNEMP,EP_HBURD,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,...,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES,Homicide Rate Per 100k
0,29.850746,5.905970,22.014925,15.489552,9.804478,19.061194,21.622388,19.011940,6.911940,0.822388,...,21.141791,1.613433,6.574627,2.894030,0.693924,0.625413,0.674607,0.531418,0.671582,12.297272
1,20.593333,7.893333,17.406667,8.780000,15.680000,14.456667,23.646667,14.080000,6.016667,1.970000,...,5.250000,10.753333,21.553333,9.890000,0.478500,0.358120,0.832597,0.805073,0.604770,5.589006
2,28.013333,7.146667,22.926667,13.900000,11.333333,21.880000,22.106667,16.086667,6.273333,4.686667,...,20.173333,5.913333,5.826667,3.180000,0.749767,0.752953,0.839607,0.787700,0.829213,6.658335
3,31.020000,5.973333,22.014667,13.812000,8.330667,19.936000,22.144000,20.978667,6.780000,1.081333,...,16.174667,2.529333,6.849333,3.348000,0.673488,0.725617,0.560131,0.593488,0.690255,9.521818
4,21.489655,6.765517,28.112069,14.094828,6.887931,18.608621,21.793103,13.675862,5.603448,6.181034,...,7.115517,5.760345,5.698276,3.327586,0.631514,0.551059,0.802922,0.709186,0.690778,4.294205


,EP_POV150,EP_UNEMP,EP_HBURD,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,...,EP_MUNIT,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
0,29.850746,5.905970,22.014925,15.489552,9.804478,19.061194,21.622388,19.011940,6.911940,0.822388,...,3.277612,21.141791,1.613433,6.574627,2.894030,0.693924,0.625413,0.674607,0.531418,0.671582
1,20.593333,7.893333,17.406667,8.780000,15.680000,14.456667,23.646667,14.080000,6.016667,1.970000,...,5.036667,5.250000,10.753333,21.553333,9.890000,0.478500,0.358120,0.832597,0.805073,0.604770
2,28.013333,7.146667,22.926667,13.900000,11.333333,21.880000,22.106667,16.086667,6.273333,4.686667,...,4.800000,20.173333,5.913333,5.826667,3.180000,0.749767,0.752953,0.839607,0.787700,0.829213
3,31.020000,5.973333,22.014667,13.812000,8.330667,19.936000,22.144000,20.978667,6.780000,1.081333,...,3.037333,16.174667,2.529333,6.849333,3.348000,0.673488,0.725617,0.560131,0.593488,0.690255
4,21.489655,6.765517,28.112069,14.094828,6.887931,18.608621,21.793103,13.675862,5.603448,6.181034,...,9.300000,7.115517,5.760345,5.698276,3.327586,0.631514,0.551059,0.802922,0.709186,0.690778


,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
0,0.693924,0.625413,0.674607,0.531418,0.671582
1,0.478500,0.358120,0.832597,0.805073,0.604770
2,0.749767,0.752953,0.839607,0.787700,0.829213
3,0.673488,0.725617,0.560131,0.593488,0.690255
4,0.631514,0.551059,0.802922,0.709186,0.690778


,EP_POV150,EP_UNEMP,EP_HBURD,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,EP_MINRTY,EP_MUNIT,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ
0,29.850746,5.905970,22.014925,15.489552,9.804478,19.061194,21.622388,19.011940,6.911940,0.822388,36.023881,3.277612,21.141791,1.613433,6.574627,2.894030
1,20.593333,7.893333,17.406667,8.780000,15.680000,14.456667,23.646667,14.080000,6.016667,1.970000,54.526667,5.036667,5.250000,10.753333,21.553333,9.890000
2,28.013333,7.146667,22.926667,13.900000,11.333333,21.880000,22.106667,16.086667,6.273333,4.686667,50.940000,4.800000,20.173333,5.913333,5.826667,3.180000
3,31.020000,5.973333,22.014667,13.812000,8.330667,19.936000,22.144000,20.978667,6.780000,1.081333,26.834667,3.037333,16.174667,2.529333,6.849333,3.348000
4,21.489655,6.765517,28.112069,14.094828,6.887931,18.608621,21.793103,13.675862,5.603448,6.181034,48.439655,9.300000,7.115517,5.760345,5.698276,3.327586


#MODEL SELECTION BASELINE

In [175]:
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import time,sys
from io import StringIO

#BASELINE training on unscaled data!
#split data
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)


#linear regression. NOTE: Linear regression has no hyperparameters, so no CV is needed
lin_reg = LinearRegression()

t0 = time.time()
%memit lin_reg.fit(X_train, y_train)
t1 = time.time()
base_lin_train_time = t1 - t0

t0 = time.time()
base_y_pred_lin = lin_reg.predict(X_test)
t1 = time.time()
base_lin_infer_time = t1 - t0
#SVR
#Cross-validate SVR with GRIDSEARCH
svr_param_grid = {'C': np.linspace(0.1, 10, 10), #higher C means less tolerance, lower means more tolerance
              'kernel': ['linear', 'rbf', 'poly'],
              'degree': [2, 3, 4,5,6],
              'epsilon':np.linspace(0.001, 0.1, 10)}

t0 = time.time()
grid_search = GridSearchCV(SVR(), svr_param_grid, n_jobs=-1,cv=5, verbose=True, scoring = "neg_root_mean_squared_error")
grid_search.fit(X_train, y_train)
t1 = time.time()
base_svr_train_time = t1 - t0

svr = grid_search.best_estimator_

t0 = time.time()
base_y_pred_svr = svr.predict(X_test)
t1 = time.time()
base_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost
"""
n_estimators: The number of trees in the ensemble, often increased until no further improvements are seen.
max_depth: The maximum depth of each tree, often values are between 1 and 10.
eta: The learning rate used to weight each model, often set to small values such as 0.3, 0.1, 0.01, or smaller.
subsample: The number of samples (rows) used in each tree, set to a value between 0 and 1, often 1.0 to use all samples.
colsample_bytree: Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 to use all features.
"""
XG_param_grid = {'n_estimators': [300,100,500],
              'n_jobs':[-1],
              'gamma': [0,1,1.5],
              'max_depth': [5,8,12],
              'eta': [0.1,0.3,0.05],
              'subsample':[1],
              'colsample_bytree':[0.75],
              'reg_lambda':[0.5,1],
              'reg_alpha':[3,2.5],
              }

grid_search = GridSearchCV(xgb.XGBRegressor(), XG_param_grid, cv=5, n_jobs=-1,scoring = "neg_root_mean_squared_error", verbose=True)
t0 = time.time()
grid_search = grid_search.fit(X_train, y_train)
t1 = time.time()
base_xgb_train_time = t1 - t0

xgb_reg = grid_search.best_estimator_

t0 = time.time()
base_y_pred_xgb = xgb_reg.predict(X_test)
t1= time.time()
base_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)

peak memory: 521.80 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits
Best SVR parameters: {'C': 0.1, 'degree': 5, 'epsilon': 0.001, 'kernel': 'poly'}
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}


In [176]:
#evaluate baseline regression performances
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
#lin reg
lin_reg_mse = mean_squared_error(y_test, base_y_pred_lin)
lin_reg_r2 = r2_score(y_test, base_y_pred_lin)

train_lin_reg_mse = mean_squared_error(y_train, lin_reg.predict(X_train))

#svr
svr_mse = mean_squared_error(y_test, base_y_pred_svr)
svr_r2 = r2_score(y_test, base_y_pred_svr)

train_svr_mse = mean_squared_error(y_train, svr.predict(X_train))


#xgb
xgb_mse = mean_squared_error(y_test, base_y_pred_xgb)
xgb_r2 = r2_score(y_test, base_y_pred_xgb)

train_xgb_mse = mean_squared_error(y_train, xgb_reg.predict(X_train))

#print errors
print("TRAINING VALIDATION")
print(f"Linear Regression Training MSE: {train_lin_reg_mse}")
print(f"SVR Training MSE: {train_svr_mse}")
print(f"XGBoost Training MSE: {train_xgb_mse}")

print("TESTING VALIDATION")
print(f"Linear Regression MSE: {lin_reg_mse}")
print(f"Linear Regression R^2: {lin_reg_r2}")
print(f"SVR MSE: {svr_mse}")
print(f"SVR R^2: {svr_r2}")
print(f"XGBoost MSE: {xgb_mse}")
print(f"XGBoost R^2: {xgb_r2}")

#training times
print(f"Linear Regression Training Time: {base_lin_train_time} seconds")
print(f"Base SVR Training Time: {base_svr_train_time} seconds")
print(f"Base XGBoost Training Time: {base_xgb_train_time} seconds")


TRAINING VALIDATION
Linear Regression Training MSE: 1.8579327940238386
SVR Training MSE: 3.3779118553893865
XGBoost Training MSE: 0.9878889299335096
TESTING VALIDATION
Linear Regression MSE: 6.021257024000639
Linear Regression R^2: 0.6646905299320609
SVR MSE: 6.813620428609875
SVR R^2: 0.6205656981499719
XGBoost MSE: 5.910049667541645
XGBoost R^2: 0.6708834028842183
Linear Regression Training Time: 0.4662129878997803 seconds
Base SVR Training Time: 4.946299314498901 seconds
Base XGBoost Training Time: 2.923881769180298 seconds


##EXPERIMENT 1: FEATURE SCALING

In [177]:
#normalize using minmax()
#standard scaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#SCALE ONLY THE FEATURES!
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#TRAINING ON SCALED DATA!
#linear regression. NOTE: Linear regression has no hyperparameters, so no CV is needed
scale_lin_reg = LinearRegression()

t0 = time.time()
%memit scale_lin_reg.fit(X_train, y_train)
t1 = time.time()
scale_lin_train_time = t1 - t0

t0 = time.time()
scale_y_pred_lin = scale_lin_reg.predict(X_test)
t1 = time.time()
scale_lin_infer_time = t1 - t0

#SVR
grid_search = GridSearchCV(SVR(), svr_param_grid, cv=5, n_jobs=-1, verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
scale_svr_train_time = t1 - t0

scale_svr = grid_search.best_estimator_

t0 = time.time()
scale_y_pred_svr = scale_svr.predict(X_test)
t1 = time.time()
scale_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), XG_param_grid, n_jobs=-1,cv=5, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
scale_xgb_train_time = t1 - t0

scale_xgb = grid_search.best_estimator_

t0 = time.time()
scale_y_pred_xgb = scale_xgb.predict(X_test)
t1 = time.time()
scale_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)

peak memory: 521.80 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits
peak memory: 521.80 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 2.3000000000000003, 'degree': 2, 'epsilon': 0.001, 'kernel': 'linear'}
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
peak memory: 521.80 MiB, increment: 0.00 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}


In [178]:
#evaluate baseline regression performances
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
#lin reg
scale_lin_reg_mse = mean_squared_error(y_test, scale_y_pred_lin)
scale_lin_reg_r2 = r2_score(y_test, scale_y_pred_lin)
train_scale_lin_reg_mse = mean_squared_error(y_train, scale_lin_reg.predict(X_train))


#svr
scale_svr_mse = mean_squared_error(y_test, scale_y_pred_svr)
scale_svr_r2 = r2_score(y_test, scale_y_pred_svr)
train_scale_svr_mse = mean_squared_error(y_train, scale_svr.predict(X_train))


#xgb
scale_xgb_mse = mean_squared_error(y_test, scale_y_pred_xgb)
scale_xgb_r2 = r2_score(y_test, scale_y_pred_xgb)
train_scale_xgb_mse = mean_squared_error(y_train, scale_xgb.predict(X_train))

#print errors
print("TRAINING VALIDATION")
print(f"Linear Regression Training MSE: {train_scale_lin_reg_mse}")
print(f"SVR Training MSE: {train_scale_svr_mse}")
print(f"XGBoost Training MSE: {train_scale_xgb_mse}")

print("TESTING VALIDATION")
print(f"Linear Regression MSE: {scale_lin_reg_mse}")
print(f"Linear Regression R^2: {scale_lin_reg_r2}")
print(f"SVR MSE: {scale_svr_mse}")
print(f"SVR R^2: {scale_svr_r2}")
print(f"XGBoost MSE: {scale_xgb_mse}")
print(f"XGBoost R^2: {scale_xgb_r2}")

#training times
print(f"Scaled Linear Regression Training Time: {scale_lin_train_time} seconds")
print(f"Scaled SVR Training Time: {scale_svr_train_time} seconds")
print(f"Scaled XGBoost Training Time: {scale_xgb_train_time} seconds")


TRAINING VALIDATION
Linear Regression Training MSE: 1.8579327940238386
SVR Training MSE: 6.436057875193771
XGBoost Training MSE: 0.9878889299335096
TESTING VALIDATION
Linear Regression MSE: 6.0212570240006364
Linear Regression R^2: 0.6646905299320611
SVR MSE: 5.340416141768769
SVR R^2: 0.7026049379222508
XGBoost MSE: 5.910049667541645
XGBoost R^2: 0.6708834028842183
Scaled Linear Regression Training Time: 0.4677116870880127 seconds
Scaled SVR Training Time: 1.8613686561584473 seconds
Scaled XGBoost Training Time: 2.731527090072632 seconds


In [179]:
"""OBSERVATION:
Scaling the data appears to reduce training times with GridCV noticeably,
splitting XGBoose training time in half.
MSEs appears to be unchanged however, except for SVR, which improved.
Also, SVR chose a linear kernel, rather than a polynomial one.
"""

'OBSERVATION:\nScaling the data appears to reduce training times with GridCV noticeably,\nsplitting XGBoose training time in half.\nMSEs appears to be unchanged however!\nAlso, SVR choose a linear kernel, rather than a polynomial one.\n'

##EXPERIMENT 2: Generate new features
• Polynomial features (2nd and 3rd order)


In [180]:
#2ND ORDER
#generate polynomial features derived from X data
from sklearn.preprocessing import PolynomialFeatures

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#SCALE ONLY THE FEATURES!
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)

#degree=2 for 2nd order features
poly_features = PolynomialFeatures(degree=2, include_bias=False)

#fit and transform the features
X_train = poly_features.fit_transform(X_train)
X_test = poly_features.transform(X_test)

print(X_train.shape)

#linear regression
poly_lin_reg = LinearRegression()

t0 = time.time()
%memit poly_lin_reg.fit(X_train, y_train)
t1 = time.time()
poly_lin_train_time = t1 - t0

t0 = time.time()
poly_y_pred_lin = poly_lin_reg.predict(X_test)
t1 = time.time()
poly_lin_infer_time = t1 - t0

#SVR
grid_search = GridSearchCV(SVR(), svr_param_grid, cv=5, n_jobs=-1,verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
poly_svr_train_time = t1 - t0

poly_svr = grid_search.best_estimator_

t0 = time.time()
poly_y_pred_svr = poly_svr.predict(X_test)
t1 = time.time()
poly_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), XG_param_grid, cv=5, n_jobs=-1, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
poly_xgb_train_time = t1 - t0

poly_xgb = grid_search.best_estimator_

t0 = time.time()
poly_y_pred_xgb = poly_xgb.predict(X_test)
t1 = time.time()
poly_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)



(40, 16)
(40, 152)
peak memory: 522.05 MiB, increment: 0.25 MiB
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits
peak memory: 522.05 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 7.800000000000001, 'degree': 2, 'epsilon': 0.001, 'kernel': 'linear'}
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
peak memory: 522.05 MiB, increment: 0.00 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.1, 'gamma': 1.5, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}


In [181]:
#evaluate
#lin reg
poly_lin_reg_mse = mean_squared_error(y_test, poly_y_pred_lin)
poly_lin_reg_r2 = r2_score(y_test, poly_y_pred_lin)
train_poly_lin_reg_mse = mean_squared_error(y_train, poly_lin_reg.predict(X_train))

#svr
poly_svr_mse = mean_squared_error(y_test, poly_y_pred_svr)
poly_svr_r2 = r2_score(y_test, poly_y_pred_svr)
train_poly_svr_mse = mean_squared_error(y_train, poly_svr.predict(X_train))

#xgb
poly_xgb_mse = mean_squared_error(y_test, poly_y_pred_xgb)
poly_xgb_r2 = r2_score(y_test, poly_y_pred_xgb)
train_poly_xgb_mse = mean_squared_error(y_train, poly_xgb.predict(X_train))

print("TRAINING VALIDATION")
print(f"Linear Regression Training MSE: {train_poly_lin_reg_mse}")
print(f"SVR Training MSE: {train_poly_svr_mse}")
print(f"XGBoost Training MSE: {train_poly_xgb_mse}")


print("TESTING VALIDATION")
print(f"Linear Regression MSE: {poly_lin_reg_mse}")
print(f"Linear Regression R^2: {poly_lin_reg_r2}")
print(f"SVR MSE: {poly_svr_mse}")
print(f"SVR R^2: {poly_svr_r2}")
print(f"XGBoost MSE: {poly_xgb_mse}")
print(f"XGBoost R^2: {poly_xgb_r2}")

print(f"Poly Linear Regression Training Time: {poly_lin_train_time} seconds")
print(f"Poly SVR Training Time: {poly_svr_train_time} seconds")
print(f"Poly XGBoost Training Time: {poly_xgb_train_time} seconds")


TRAINING VALIDATION
Linear Regression Training MSE: 6.214375040394961e-29
SVR Training MSE: 0.9692832948752294
XGBoost Training MSE: 0.811391216941027
TESTING VALIDATION
Linear Regression MSE: 15.811879945458458
Linear Regression R^2: 0.11947404600794098
SVR MSE: 7.388037075403568
SVR R^2: 0.5885778024885069
XGBoost MSE: 5.490413432665133
XGBoost R^2: 0.6942519458606913
Poly Linear Regression Training Time: 0.5081236362457275 seconds
Poly SVR Training Time: 1.975726842880249 seconds
Poly XGBoost Training Time: 5.559717416763306 seconds


In [182]:
"""OBSERVATION:
Introducing a polynomial feature appears to significantly worsen the performance
of the linear regression model - this is expected since the purpose is to
introduce non-linearity to the data. This results in linear regression highly overfitting the train data, creating a too complex model
especially when there is a small sample size of 40 combined with 152 features created by the polynomial transformation.

Polynomial features can introduce high multicollinearity (strong correlation among the features).
This can make it difficult for linear regression to estimate the
coefficients accurately and can lead to worse predictions.

For 2nd order polynomials,
XGBoost performed the best out of the three models.
This may be because XGBoost captures more complex relationships within the data
better than simple linear regressions, showing its superiority in high-dimensional data.

Choosing 3rd order polynomial however, significantly improved SVR testing error,
making it the best model, while XGBoost showed signs of overfitting.

2nd order results:
TRAINING VALIDATION
Linear Regression Training MSE: 6.214375040394961e-29
SVR Training MSE: 0.9692832948752294
XGBoost Training MSE: 0.811391216941027
TESTING VALIDATION
Linear Regression MSE: 15.811879945458458
Linear Regression R^2: 0.11947404600794098
SVR MSE: 7.388037075403568
SVR R^2: 0.5885778024885069
XGBoost MSE: 5.490413432665133
XGBoost R^2: 0.6942519458606913
Poly Linear Regression Training Time: 0.40543150901794434 seconds
Poly SVR Training Time: 1.9447619915008545 seconds
Poly XGBoost Training Time: 5.509463310241699 seconds

3rd order results
TRAINING VALIDATION
Linear Regression Training MSE: 1.0319903014004565e-28
SVR Training MSE: 1.0182558407620799
XGBoost Training MSE: 0.5436830734271374
TESTING VALIDATION
Linear Regression MSE: 16.242248176341487
Linear Regression R^2: 0.0955078637213751
SVR MSE: 6.297163643151013
SVR R^2: 0.6493259471071144
XGBoost MSE: 6.934081755189215
XGBoost R^2: 0.6138574936308825
Poly Linear Regression Training Time: 0.4049391746520996 seconds
Poly SVR Training Time: 2.2321090698242188 seconds
Poly XGBoost Training Time: 25.893834590911865 seconds
"""

'OBSERVATION:\nIntroducing a polynomial feature appears to significantly worsen the performance\nof the linear regression model - this is expected since the purpose is to\nintroduce non-linearity to the data. This results in linear regression highly overfitting the train data, creating a too complex model\nespecially when there is a small sample size of 40 combined with 152 features created by the polynomial transformation.\n\nPolynomial features can introduce high multicollinearity (strong correlation among the features).\nThis can make it difficult for linear regression to estimate the\ncoefficients accurately and can lead to worse predictions.\n\nFor 2nd order polynomials,\nXGBoost performed the best out of the three models.\nThis may be because XGBoost captures more complex relationships within the data\nbetter than simple linear regressions, showing its superiority in high-dimensional data.\n\nChoosing 3rd order polynomial however, significantly improved SVR testing error,\nmaking

##EXPERIMENT 3: FEATURE TRANSFORMATION

In [183]:
#USE PCA
from sklearn.decomposition import PCA

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#apply scaling
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#apply pca
pca = PCA(n_components=0.9)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

#lin reg
pca_lin_reg = LinearRegression()

t0 = time.time()
%memit pca_lin_reg.fit(X_train, y_train)
t1 = time.time()
pca_lin_train_time = t1 - t0

t0 = time.time()
pca_y_pred_lin = pca_lin_reg.predict(X_test)
t1 = time.time()
pca_lin_infer_time = t1 - t0

#SVR
grid_search = GridSearchCV(SVR(), svr_param_grid, cv=5, n_jobs=-1, verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
pca_svr_train_time = t1 - t0

pca_svr = grid_search.best_estimator_

t0 = time.time()
pca_y_pred_svr = pca_svr.predict(X_test)
t1 = time.time()
pca_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), XG_param_grid, cv=5, n_jobs=-1, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
pca_xgb_train_time = t1 - t0

pca_xgb = grid_search.best_estimator_

t0 = time.time()
pca_y_pred_xgb = pca_xgb.predict(X_test)
t1 = time.time()
pca_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)



peak memory: 522.05 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits
peak memory: 522.05 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 4.5, 'degree': 3, 'epsilon': 0.1, 'kernel': 'poly'}
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
peak memory: 522.05 MiB, increment: 0.00 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 0, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 1, 'subsample': 1}


In [184]:
#evaluate
#lin reg
pca_lin_reg_mse = mean_squared_error(y_test, pca_y_pred_lin)
pca_lin_reg_r2 = r2_score(y_test, pca_y_pred_lin)
train_pca_lin_reg_mse = mean_squared_error(y_train, pca_lin_reg.predict(X_train))

#svr
pca_svr_mse = mean_squared_error(y_test, pca_y_pred_svr)
pca_svr_r2 = r2_score(y_test, pca_y_pred_svr)
train_pca_svr_mse = mean_squared_error(y_train, pca_svr.predict(X_train))

#xgb
pca_xgb_mse = mean_squared_error(y_test, pca_y_pred_xgb)
pca_xgb_r2 = r2_score(y_test, pca_y_pred_xgb)
train_pca_xgb_mse = mean_squared_error(y_train, pca_xgb.predict(X_train))

print("TRAINING VALIDATION")
print(f"Linear Regression Training MSE: {train_pca_lin_reg_mse}")
print(f"SVR Training MSE: {train_pca_svr_mse}")
print(f"XGBoost Training MSE: {train_pca_xgb_mse}")

print("TESTING VALIDATION")
print(f"Linear Regression MSE: {pca_lin_reg_mse}")
print(f"Linear Regression R^2: {pca_lin_reg_r2}")
print(f"SVR MSE: {pca_svr_mse}")
print(f"SVR R^2: {pca_svr_r2}")
print(f"XGBoost MSE: {pca_xgb_mse}")
print(f"XGBoost R^2: {pca_xgb_r2}")

print(f"PCA Linear Regression Training Time: {pca_lin_train_time} seconds")
print(f"PCA SVR Training Time: {pca_svr_train_time} seconds")
print(f"PCA XGBoost Training Time: {pca_xgb_train_time} seconds")

print(f"PCA reduced to {pca.n_components_} components! ")

TRAINING VALIDATION
Linear Regression Training MSE: 3.3223205107630847
SVR Training MSE: 1.8612041339912593
XGBoost Training MSE: 0.30955425951736826
TESTING VALIDATION
Linear Regression MSE: 5.839949811238412
Linear Regression R^2: 0.6747870970090805
SVR MSE: 7.21511477037949
SVR R^2: 0.5982074340138566
XGBoost MSE: 5.586542281781182
XGBoost R^2: 0.688898759088085
PCA Linear Regression Training Time: 0.5001959800720215 seconds
PCA SVR Training Time: 1.9499282836914062 seconds
PCA XGBoost Training Time: 2.503145694732666 seconds
PCA reduced to 6 components! 


In [185]:
"""OBSERVATION:
Reducing PCA to capture 90% of variance reduced the features to 6 components (out of 18).
However, this also appears to raise MSE for XGBoost, while lowering SVR, and Linear regression.
Further experimentation with XGBoost parameters may be needed.
"""

'OBSERVATION:\nReducing PCA to capture 90% of variance reduced the features to 6 components (out of 18).\nHowever, this also appears to raise MSE for XGBoost, while lowering SVR, and Linear regression.\nFurther experimentation with XGBoost parameters may be needed.\n'

##EXPERIMENT 4: HANDLING NOISE
Introduce synthetic noise:

• Add a random continuous feature (you can use random.Generator.uniform)

• Add a random discrete categorical feature

In [186]:
#Introduce synthetic noise:
#Add a random continuous feature using random.Generator.uniform

noise = np.random.rand(len(data))

#random discrete categorical feature, 2 classes
random_categories = np.random.randint(0, 2, size=len(data))
data['RANDOM_CAT'] = random_categories
data['NOISE'] = noise
#display(data)

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#apply scaling
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#lin reg
noise_lin_reg = LinearRegression()

t0 = time.time()
%memit noise_lin_reg.fit(X_train, y_train)
t1 = time.time()
noise_lin_train_time = t1 - t0

t0 = time.time()
noise_y_pred_lin = noise_lin_reg.predict(X_test)
t1 = time.time()
noise_lin_infer_time = t1 - t0

#SVR
grid_search = GridSearchCV(SVR(), svr_param_grid, cv=5, n_jobs=-1, verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
noise_svr_train_time = t1 - t0

noise_svr = grid_search.best_estimator_

t0 = time.time()
noise_y_pred_svr = noise_svr.predict(X_test)
t1 = time.time()
noise_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), XG_param_grid, cv=5, n_jobs=-1, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
noise_xgb_train_time = t1 - t0

noise_xgb = grid_search.best_estimator_

t0 = time.time()
noise_y_pred_xgb = noise_xgb.predict(X_test)
t1 = time.time()
noise_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)



peak memory: 522.05 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits
peak memory: 522.05 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 3.4000000000000004, 'degree': 2, 'epsilon': 0.001, 'kernel': 'linear'}
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
peak memory: 522.05 MiB, increment: 0.00 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 8, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 1, 'subsample': 1}


In [187]:
#evaluate
#lin reg
noise_lin_reg_mse = mean_squared_error(y_test, noise_y_pred_lin)
noise_lin_reg_r2 = r2_score(y_test, noise_y_pred_lin)
train_noise_lin_reg_mse = mean_squared_error(y_train, noise_lin_reg.predict(X_train))

#svr
noise_svr_mse = mean_squared_error(y_test, noise_y_pred_svr)
noise_svr_r2 = r2_score(y_test, noise_y_pred_svr)
train_noise_svr_mse = mean_squared_error(y_train, noise_svr.predict(X_train))

#xgb
noise_xgb_mse = mean_squared_error(y_test, noise_y_pred_xgb)
noise_xgb_r2 = r2_score(y_test, noise_y_pred_xgb)
train_noise_xgb_mse = mean_squared_error(y_train, noise_xgb.predict(X_train))

print("TRAINING VALIDATION")
print(f"Linear Regression Training MSE: {train_noise_lin_reg_mse}")
print(f"SVR Training MSE: {train_noise_svr_mse}")
print(f"XGBoost Training MSE: {train_noise_xgb_mse}")

print("TESTING VALIDATION")
print(f"Linear Regression MSE: {noise_lin_reg_mse}")
print(f"Linear Regression R^2: {noise_lin_reg_r2}")
print(f"SVR MSE: {noise_svr_mse}")
print(f"SVR R^2: {noise_svr_r2}")
print(f"XGBoost MSE: {noise_xgb_mse}")
print(f"XGBoost R^2: {noise_xgb_r2}")

print(f"Noisy Linear Regression Training Time: {noise_lin_train_time} seconds")
print(f"Noisy SVR Training Time: {noise_svr_train_time} seconds")
print(f"Noisy XGBoost Training Time: {noise_xgb_train_time} seconds")

#SVR appears most robust to noisy data!


TRAINING VALIDATION
Linear Regression Training MSE: 1.6413816054918349
SVR Training MSE: 5.233692401214437
XGBoost Training MSE: 0.7895197633356645
TESTING VALIDATION
Linear Regression MSE: 7.959046258848479
Linear Regression R^2: 0.556779660349471
SVR MSE: 5.617370160145802
SVR R^2: 0.6871820279277049
XGBoost MSE: 7.449110418871594
XGBoost R^2: 0.5851767733758291
Noisy Linear Regression Training Time: 0.46941590309143066 seconds
Noisy SVR Training Time: 1.8483541011810303 seconds
Noisy XGBoost Training Time: 2.725550413131714 seconds


##Model Efficiency

In [188]:
#TIME COMPARISONS
time_df = pd.DataFrame(columns=['Model Variant', 'Training Time', 'Inference Time'])

#Linear Regression
time_df.loc[len(time_df)] = ['BASE Linear Regression', base_lin_train_time, base_lin_infer_time]
time_df.loc[len(time_df)] = ['SCALED Linear Regression', scale_lin_train_time, scale_lin_infer_time]
time_df.loc[len(time_df)] = ['POLY Linear Regression', poly_lin_train_time, poly_lin_infer_time]
time_df.loc[len(time_df)] = ['PCA Linear Regression', pca_lin_train_time, pca_lin_infer_time]
time_df.loc[len(time_df)] = ['NOISY Linear Regression', noise_lin_train_time, noise_lin_infer_time]

#SVR
time_df.loc[len(time_df)] = ['BASE SVR', base_svr_train_time, base_svr_infer_time]
time_df.loc[len(time_df)] = ['SCALED SVR', scale_svr_train_time, scale_svr_infer_time]
time_df.loc[len(time_df)] = ['POLY SVR', poly_svr_train_time, poly_svr_infer_time]
time_df.loc[len(time_df)] = ['PCA SVR', pca_svr_train_time, pca_svr_infer_time]
time_df.loc[len(time_df)] = ['NOISY SVR', noise_svr_train_time, noise_svr_infer_time]

#XGBoost
time_df.loc[len(time_df)] = ['BASE XGBoost', base_xgb_train_time, base_xgb_infer_time]
time_df.loc[len(time_df)] = ['SCALED XGBoost', scale_xgb_train_time, scale_xgb_infer_time]
time_df.loc[len(time_df)] = ['POLY XGBoost', poly_xgb_train_time, poly_xgb_infer_time]
time_df.loc[len(time_df)] = ['PCA XGBoost', pca_xgb_train_time, pca_xgb_infer_time]
time_df.loc[len(time_df)] = ['NOISY XGBoost', noise_xgb_train_time, noise_xgb_infer_time]

display(time_df)

"""
If required to deploy a model, which trade-offs are necessary?

If speed is important: Linear regression offers the fastest speeds, and decent
accuracy for this dataset (except for when adding Polynomial features!).

If accuracy is important: SVR outperformed linear and XGBoost in some cases,
although linear had good accuracy as well, along with its speed.
XGBoost requires much more hyperparameter experimentation to be considered as the best.

If interpretability is important: Linear Regression is the preferred choice,
as it offers a clearer understanding of feature importance and relationships.
"""


,Model Variant,Training Time,Inference Time
0,BASE Linear Regression,0.466213,0.000973
1,SCALED Linear Regression,0.467712,0.000452
2,POLY Linear Regression,0.508124,0.000346
3,PCA Linear Regression,0.500196,0.000287
4,NOISY Linear Regression,0.469416,0.000291
5,BASE SVR,4.946299,0.001350
6,SCALED SVR,1.861369,0.000886
7,POLY SVR,1.975727,0.000829
8,PCA SVR,1.949928,0.000865
9,NOISY SVR,1.848354,0.000809


'\nIf required to deploy a model, which trade-offs are necessary?\n\nIf speed is important: Linear regression offers the fastest speeds, and decent\naccuracy for this dataset (except for when adding Polynomial features!).\n\nIf accuracy is important: SVR outperformed linear and XGBoost in some cases,\nalthough linear had good accuracy as well, along with its speed.\nXGBoost requires much more hyperparameter experimentation to be considered as the best.\n\nIf interpretability is important: Linear Regression is the preferred choice,\nas it offers a clearer understanding of feature importance and relationships.\n'

##Summarization of findings


In [191]:

#SUMMARIZATION TABLE
mse_df = pd.DataFrame(columns=['Experiment','Model Variant', 'Train MSE', 'Test MSE'])
#linear reg errors
mse_df.loc[len(mse_df)] = ['BASE','Linear Regression', train_lin_reg_mse, lin_reg_mse]
mse_df.loc[len(mse_df)] = ['SCALED', 'Linear Regression', train_scale_lin_reg_mse, scale_lin_reg_mse]
mse_df.loc[len(mse_df)] = ['POLY', 'Linear Regression', train_poly_lin_reg_mse, poly_lin_reg_mse]
mse_df.loc[len(mse_df)] = ['PCA', 'Linear Regression', train_pca_lin_reg_mse, pca_lin_reg_mse]
mse_df.loc[len(mse_df)] = ['NOISY', 'Linear Regression', train_noise_lin_reg_mse, noise_lin_reg_mse]

# SVR
mse_df.loc[len(mse_df)] = ['BASE', 'SVR', train_svr_mse, svr_mse]
mse_df.loc[len(mse_df)] = ['SCALED', 'SVR', train_scale_svr_mse, scale_svr_mse]
mse_df.loc[len(mse_df)] = ['POLY','SVR', train_poly_svr_mse, poly_svr_mse]
mse_df.loc[len(mse_df)] = ['PCA', 'SVR', train_pca_svr_mse, pca_svr_mse]
mse_df.loc[len(mse_df)] = ['NOISY', 'SVR', train_noise_svr_mse, noise_svr_mse]

# XGBoost
mse_df.loc[len(mse_df)] = ['BASE', 'XGBoost', train_xgb_mse, xgb_mse]
mse_df.loc[len(mse_df)] = ['SCALED', 'XGBoost', train_scale_xgb_mse, scale_xgb_mse]
mse_df.loc[len(mse_df)] = ['POLY', 'XGBoost', train_poly_xgb_mse, poly_xgb_mse]
mse_df.loc[len(mse_df)] = ['PCA', 'XGBoost', train_pca_xgb_mse, pca_xgb_mse]
mse_df.loc[len(mse_df)] = ['NOISY', 'XGBoost', train_noise_xgb_mse, noise_xgb_mse]


#display, ordered by experiemnt
display(mse_df.sort_values(by=['Experiment']))

#display only base
#display(mse_df[mse_df['Experiment'] == 'BASE'])

print(f"Lowest Train MSE: {mse_df.loc[mse_df['Train MSE'].idxmin()]['Train MSE']} {mse_df.loc[mse_df['Train MSE'].idxmin()]['Model Variant']} {mse_df.loc[mse_df['Train MSE'].idxmin()]['Experiment']}")
print(f"Highest Train MSE: {mse_df.loc[mse_df['Train MSE'].idxmax()]['Train MSE']} {mse_df.loc[mse_df['Train MSE'].idxmax()]['Model Variant']} {mse_df.loc[mse_df['Train MSE'].idxmax()]['Experiment']}")

print(f"Lowest Test MSE: {mse_df.loc[mse_df['Test MSE'].idxmin()]['Test MSE']} {mse_df.loc[mse_df['Test MSE'].idxmin()]['Model Variant']} {mse_df.loc[mse_df['Test MSE'].idxmin()]['Experiment']}")
print(f"Highest Test MSE: {mse_df.loc[mse_df['Test MSE'].idxmax()]['Test MSE']} {mse_df.loc[mse_df['Test MSE'].idxmax()]['Model Variant']} {mse_df.loc[mse_df['Test MSE'].idxmax()]['Experiment']}")

,Experiment,Model Variant,Train MSE,Test MSE
0,BASE,Linear Regression,1.857933e+00,6.021257
5,BASE,SVR,3.377912e+00,6.813620
10,BASE,XGBoost,9.878889e-01,5.910050
4,NOISY,Linear Regression,1.641382e+00,7.959046
9,NOISY,SVR,5.233692e+00,5.617370
14,NOISY,XGBoost,7.895198e-01,7.449110
3,PCA,Linear Regression,3.322321e+00,5.839950
8,PCA,SVR,1.861204e+00,7.215115
13,PCA,XGBoost,3.095543e-01,5.586542
2,POLY,Linear Regression,6.214375e-29,15.811880


Lowest Train MSE: 6.214375040394961e-29 Linear Regression POLY
Highest Train MSE: 6.436057875193771 SVR SCALED
Lowest Test MSE: 5.340416141768769 SVR SCALED
Highest Test MSE: 15.811879945458458 Linear Regression POLY


##Final model choice

It appears that SVR with a linear kernal on scaled features achieved the lowest MSE compared to other model variants - therefore, this is my recomended model for the given dataset. This model effectively captured the general pattern within the dataset without excessive overfitting, leading to improved predictive performance.

The fact that only scaled data was required to achieve the best results suggests that the dataset exhibits a linear relationship between the features and the target. Simple linear models, like SVR with a linear kernel, is sufficient to capture the underlying patterns, while XGBoost likely overfit the data. More complex feature transformations does not appear provide significant improvements.

##Proposed Further Refinements
If more time, and patience, were available, a more extensive GridSearch over a wider range of values could potentially yield better results, especially for XGBoost which tended to overfit the data. Using techniques like RandomizedSearchCV or Bayesian Optimization would alternatively allow for a more efficient exploration of the hyperparameter space.

Also, due to only 50 samples (50 states), it may be better to incorporate more data from additional years, or elect to analyze county data.

Experimenting with other models, particularly  Ensemble Methods could potentially improve overall accuracy and stability.

